<a href="https://colab.research.google.com/github/2003vivek/2003vivek/blob/main/beer_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/train.csv')
df.sample(3)

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,review/timeUnix,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
12972,38547,5.5,25468,568,Palone,Euro Dark Lager,4.5,4.0,4.0,4.0,4.0,I do not know the style this really is so I wi...,"{'min': 52, 'hour': 21, 'mday': 3, 'sec': 29, ...",1223070749,NaN,NaN,NaN,Male,morimech
2330,48963,10.0,20470,394,Stoudt's Double IPA (India Pale Ale),American Double / Imperial IPA,3.5,4.0,4.0,4.0,3.5,12 oz. bottle tried 12 Oct 2008. Trade from un...,"{'min': 40, 'hour': 0, 'mday': 1, 'sec': 36, '...",1233448836,NaN,NaN,NaN,NaN,htomsirveaux
26929,25616,6.6,16074,1199,Founders Red's Rye PA,Rye Beer,4.0,4.0,4.5,4.0,4.0,"A: A ruby, dark copper color with a large fair...","{'min': 49, 'hour': 19, 'mday': 29, 'sec': 4, ...",1291060144,NaN,NaN,NaN,NaN,SickGnar


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37500 entries, 0 to 37499
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              37500 non-null  int64  
 1   beer/ABV           37500 non-null  float64
 2   beer/beerId        37500 non-null  int64  
 3   beer/brewerId      37500 non-null  int64  
 4   beer/name          37500 non-null  object 
 5   beer/style         37500 non-null  object 
 6   review/appearance  37500 non-null  float64
 7   review/aroma       37500 non-null  float64
 8   review/overall     37500 non-null  float64
 9   review/palate      37500 non-null  float64
 10  review/taste       37500 non-null  float64
 11  review/text        37490 non-null  object 
 12  review/timeStruct  37500 non-null  object 
 13  review/timeUnix    37500 non-null  int64  
 14  user/ageInSeconds  7856 non-null   float64
 15  user/birthdayRaw   7856 non-null   object 
 16  user/birthdayUnix  785

In [7]:
df.corr()['review/overall']

<ipython-input-7-92de16046b77>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['review/overall']


index                0.054493
beer/ABV             0.197005
beer/beerId         -0.021981
beer/brewerId       -0.084932
review/appearance    0.498733
review/aroma         0.616117
review/overall       1.000000
review/palate        0.697220
review/taste         0.785220
review/timeUnix      0.065383
user/ageInSeconds   -0.045642
user/birthdayUnix    0.045642
Name: review/overall, dtype: float64

In [8]:
new_df=df[['beer/ABV','review/appearance','review/aroma','review/palate','review/taste','review/text','review/overall']]

In [9]:
new_df.sample(6)

,beer/ABV,review/appearance,review/aroma,review/palate,review/taste,review/text,review/overall
31828,6.0,4.0,3.5,4.0,3.5,Pours a deep black with a one inch head that f...,4.5
31761,11.2,4.5,4.5,5.0,5.0,"Pours pitch black into a snifter. Thin, wispy ...",5.0
6938,6.6,4.0,4.5,4.0,4.5,Translucent caramel color with a HUGE foamy wh...,3.5
37414,6.6,4.5,4.0,4.0,5.0,Black color. Medium body---not syrup thick. Ar...,4.5
23985,6.6,3.0,4.0,3.5,4.5,Pours a beautiful raspberry amber color with a...,5.0
28243,8.5,4.0,4.0,4.5,4.5,Bottle poured into tulip glass. Second tasting...,4.0


In [10]:
new_df.isnull().sum()

beer/ABV              0
review/appearance     0
review/aroma          0
review/palate         0
review/taste          0
review/text          10
review/overall        0
dtype: int64

In [11]:
new_df=new_df.dropna()

In [12]:
new_df.isnull().sum()

beer/ABV             0
review/appearance    0
review/aroma         0
review/palate        0
review/taste         0
review/text          0
review/overall       0
dtype: int64

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,accuracy_score

In [14]:
text=new_df['review/text']
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(text)

In [15]:
x_numericals=new_df.drop(['review/text','review/overall'],axis=1)

In [16]:
from scipy.sparse import hstack
data=hstack((x_numericals,X_text))
y=new_df['review/overall']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

In [18]:
model=RandomForestRegressor(random_state=42)

In [ ]:
model.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
mse=mean_squared_error(y_test,y_pred)
print(f'mean squared error is : {mse}')

mean squared error is : 0.15478542611363033


In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(y_test,y_pred)
r2

0.6930328238468451

In [ ]:
import pickle
with open('model_pickle','wb') as f:
  pickle.dump(model,f)

In [ ]:
text = 'pathetic'
text_vectorized = vectorizer.transform([text]).toarray().flatten()
unseen_data = [6.0, 3.0, 4.0, 3.0, 3.5] + text_vectorized.tolist()
model.predict([unseen_data])


array([3.36])

2nd model svr


In [19]:
model2=SVR(kernel='linear',C=1.0)

In [ ]:
model2.fit(X_train,y_train)

In [20]:
y_pred=model2.predict(y_test)

NotFittedError: This SVR instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
mse=mean_squared_error(y_test,y_pred)

In [ ]:
import pickle
with open('model2_pickle','wb') as f:
  pickle.dump(model2,f)

In [ ]:
text = 'This beer should have more alcohols '
text_vectorized = vectorizer.transform([text]).toarray().flatten()
unseen_data = [6.0, 3.0, 4.0, 3.0, 3.5] + text_vectorized.tolist()
model2.predict([unseen_data])
